# Пример того, как собирать деньги с помощью Payments

In [1]:
import os
import asyncio
import aiohttp
from typing import AsyncGenerator, Optional, Tuple
import json

from tg import send_message, long_pol_messages

In [6]:
async def send_invoice(**kwargs):
    url = f'https://api.telegram.org/bot{TELEGRAM_TOKEN}/sendInvoice'
    async with aiohttp.ClientSession() as session:
        async with session.post(url, json=kwargs) as response:
            if response.status!= 200:
                raise Exception(f'Failed to send message: {response.status}, {await response.text()}')
            else:
                return await response.json()


async def answer_pre_checkout_query(**kwargs):
    url = f'https://api.telegram.org/bot{TELEGRAM_TOKEN}/answerPreCheckoutQuery'
    async with aiohttp.ClientSession() as session:
        async with session.post(url, json=kwargs) as response:
            if response.status!= 200:
                raise Exception(f'Failed to send message: {response.status}, {await response.text()}')
            else:
                return await response.json()

In [ ]:
async for update in long_pol_messages():
    print(update)
    if 'message' in update:
        message = update['message']
        if 'successful_payment' in message:
            # example update
            # {'update_id': 259828801, 'message': {'message_id': 296, 'from': {'id': 87799679, 'is_bot': False, 'first_name': 'Marat', 'username': 'tsundokum', 'language_code': 'en'}, 'chat': {'id': 87799679, 'first_name': 'Marat', 'username': 'tsundokum', 'type': 'private'}, 'date': 1678524414, 'successful_payment': {'currency': 'RUB', 'total_amount': 50000, 'invoice_payload': '87799679', 'telegram_payment_charge_id': '6045619726_87799679_509653_7209207390913721716', 'provider_payment_charge_id': '2b9e53bb-000f-5000-8000-1058b15bfb60'}}}
            sp = message['successful_payment']
            print(f"Just received {sp['total_amount'] / 100:.2f} {sp['currency']}")
        else:
            button = [[{
                "text": "Поддержать проект",
                "callback_data": json.dumps({'type': 'donate_button',
                                             'update_id': message.get('update_id', -1)})
            }]]

            await send_message(chat_id=message['chat']['id'],
                               reply_to_message_id=message['message_id'],
                               text=f"Ответ на ваш запрос {message['text']}",
                               reply_markup={"inline_keyboard": button})
    elif 'callback_query' in update:
        # example update
        # {'update_id': 259828799, 'callback_query': {'id': '377096750973562467', 'from': {'id': 87799679, 'is_bot': False, 'first_name': 'Marat', 'username': 'tsundokum', 'language_code': 'en'}, 'message': {'message_id': 294, 'from': {'id': 6045619726, 'is_bot': True, 'first_name': 'Transcraber DEV', 'username': 'transcraber_dev_bot'}, 'chat': {'id': 87799679, 'first_name': 'Marat', 'username': 'tsundokum', 'type': 'private'}, 'date': 1678524395, 'reply_to_message': {'message_id': 293, 'from': {'id': 87799679, 'is_bot': False, 'first_name': 'Marat', 'username': 'tsundokum', 'language_code': 'en'}, 'chat': {'id': 87799679, 'first_name': 'Marat', 'username': 'tsundokum', 'type': 'private'}, 'date': 1678524394, 'text': 'dtyhdfuhjfg'}, 'text': 'Ответ на ваш запрос dtyhdfuhjfg', 'reply_markup': {'inline_keyboard': [[{'text': 'Поддержать проект', 'callback_data': '{"type": "donate_button", "update_id": -1}'}]]}}, 'chat_instance': '-4362863864751142394', 'data': '{"type": "donate_button", "update_id": -1}'}}
        cq = update['callback_query']
        r = await send_invoice(chat_id=cq['from']['id'], 
                               title='Пожертвование',
                               currency='rub',
                               prices=[{'label': 'Поддержка', 'amount': 500 * 100}],
                               description="Поддержка проекта",
                               payload=f"{cq['from']['id']}",
                               provider_token=os.environ['YOOKASSA_TEST'])
        print(f"{r=}")
    elif 'pre_checkout_query' in update:
        # example update
        # {'update_id': 259828800, 'pre_checkout_query': {'id': '377096751211060898', 'from': {'id': 87799679, 'is_bot': False, 'first_name': 'Marat', 'username': 'tsundokum', 'language_code': 'en'}, 'currency': 'RUB', 'total_amount': 50000, 'invoice_payload': '87799679'}}
        pcq = update['pre_checkout_query']
        res_pcq = await answer_pre_checkout_query(pre_checkout_query_id=pcq['id'], ok=True)
        print(f"{res_pcq=}")


{'update_id': 259828797, 'message': {'message_id': 292, 'from': {'id': 87799679, 'is_bot': False, 'first_name': 'Marat', 'username': 'tsundokum', 'language_code': 'en'}, 'chat': {'id': 87799679, 'first_name': 'Marat', 'username': 'tsundokum', 'type': 'private'}, 'date': 1678524186, 'successful_payment': {'currency': 'RUB', 'total_amount': 50000, 'invoice_payload': '87799679', 'telegram_payment_charge_id': '6045619726_87799679_509649_7209206392798919648', 'provider_payment_charge_id': '2b9e52d9-000f-5000-a000-1048affb60c4'}}}
Just received 500.00 RUB
{'update_id': 259828798, 'message': {'message_id': 293, 'from': {'id': 87799679, 'is_bot': False, 'first_name': 'Marat', 'username': 'tsundokum', 'language_code': 'en'}, 'chat': {'id': 87799679, 'first_name': 'Marat', 'username': 'tsundokum', 'type': 'private'}, 'date': 1678524394, 'text': 'dtyhdfuhjfg'}}
{'update_id': 259828799, 'callback_query': {'id': '377096750973562467', 'from': {'id': 87799679, 'is_bot': False, 'first_name': 'Marat', 